## Interactive Map with a Year Slider to Fetch All Qualifying Times
This
notebook
creates
an
interactive
map
that
allows
users
to
select
a
circuit
on
a
map and use
a
slider
to
choose
a
year.When
clicking
on
a
circuit, the
map
fetches
the
maximum
qualifying
times(`q1`, `q2`, `q3`)
for the specified circuit and year.


In [6]:
# Required imports for the implementation
import folium
from folium.plugins import MarkerCluster
from ipywidgets import interact, IntSlider, Output
from IPython.display import display
import requests
import pandas as pd

# Load circuit data from file or API
circuits = pd.read_csv(
    "../data/circuits.csv"
)  # Assuming circuits file contains latitude, longitude, and id fields

# Define base URL for the API
API_BASE_URL = "http://localhost:8000"  # Replace with your actual API base URL

In [7]:
# Create a function to generate the map
def generate_map(year):
    """
    Generate an interactive map for circuits with a given year.
    Clicking on a circuit displays its max qualifying times retrieved from the API.

    :param year: An integer for the selected year, passed from the slider.
    :return: Displays an interactive map object.
    """
    # Create an output widget for displaying API results
    output = Output()

    # Initialize the map centered on the average coordinates
    m = folium.Map(location=[circuits["lat"].mean(), circuits["lng"].mean()], zoom_start=2)

    # Create a marker cluster
    marker_cluster = MarkerCluster().add_to(m)

    # Add markers for each circuit
    for index, row in circuits.iterrows():
        # Extract information from the row (ensure columns exist in the CSV)
        circuit_ref = row["circuitRef"]
        circuit_name = row.get("name", "Unknown Circuit")
        lat, lng = row["lat"], row["lng"]

        # Add a marker with a clickable popup
        def add_marker_to_map():
            """
            Define the action for the marker popup when clicked.
            """

            # Fetch results from API for the circuit and year
            def fetch_results():
                try:
                    # Perform the API call
                    response = requests.get(
                        f"{API_BASE_URL}/temps_dernier_qualif_{circuit_ref}_{year}"
                    )
                    if response.status_code == 200:
                        # Parse and display the qualifying times
                        response_data = response.json()

                        # Skip marker if no valid qualifying data
                        if not (
                            response_data.get("q1")
                            or response_data.get("q2")
                            or response_data.get("q3")
                        ):
                            return None

                        display_text = (
                            f"<strong>Circuit:</strong> {circuit_name}<br>"
                            f"<strong>Year:</strong> {year}<br>"
                            f"<strong>Q1:</strong> {response_data.get('q1', 'N/A')}<br>"
                            f"<strong>Q2:</strong> {response_data.get('q2', 'N/A')}<br>"
                            f"<strong>Q3:</strong> {response_data.get('q3', 'N/A')}<br>"
                        )
                        return folium.Popup(display_text, max_width=300)
                    else:
                        # Display error message
                        return folium.Popup(
                            f"Error {response.status_code}: {response.json().get('detail', 'No details')}",
                            max_width=300,
                        )
                except Exception as e:
                    return folium.Popup(f"Error occurred while fetching data: {e}", max_width=300)

            # Fetch results and create dynamic popup
            popup = fetch_results()

            # Add marker only if valid popup is returned
            if popup:
                folium.Marker(
                    location=[lat, lng],
                    popup=popup,
                    tooltip=f"{circuit_name}",
                    color="pink",
                ).add_to(marker_cluster)

        add_marker_to_map()

    # Render the output widget (to show results below the map)
    display(output)
    return m

In [8]:
# Define an interactive function to update the map
slider = IntSlider(
    min=1950,
    max=2024,
    step=1,
    value=2000,
    description="Year",
    continuous_update=False,  # Avoid unnecessary updates
)


@interact(year=slider)
def update_map(year):
    """
    Update the map based on the selected year using the slider.
    """
    display(generate_map(year))

interactive(children=(IntSlider(value=2000, continuous_update=False, description='Year', max=2024, min=1950), …